## Getting Started with MAF - Creating and Chatting with our Agent (At AI Foundry Project Level)

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4

### Setting up the environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential

load_dotenv()
project_endpoint = os.getenv("AI_FOUNDRY_PROJECT_ENDPOINT")
model = os.getenv("AI_FOUNDRY_DEPLOYMENT_NAME")

print("Project Endpoint: ", project_endpoint)
print("Model: ", model)

### Creating the Azure AI Foundry Client and Agent

In [ ]:
from azure.ai.projects.aio import AIProjectClient
from azure.identity import DefaultAzureCredential
from agent_framework_azure_ai import AzureAIAgentClient

# creating an AzureAIProjectClient
project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=project_endpoint
)

# Creating an AzureAIAgentClient
agent_client = AzureAIAgentClient(
    project_client=project_client,
    model_deployment_name=model
)

agent = agent_client.create_agent(
    name = "BatmanAgent",
    instructions = "You are Batman, the dark knight of Gotham City."
)

### Chatting with the Agent - Non Streaming

In [ ]:
query = "Who is the Joker?"
response = await agent.run(query)
print("Agent Response:", response)


### Chatting with the Agent - Streaming

In [ ]:
# Running the agent with streaming responses
async def streaming_chat():
    async for update in agent.run_stream("Tell me something interesting about Gotham City in 10 points"):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

await streaming_chat()

### Giving our Agent an Image

![image.png](./images/joker_interrogation_scene.png)

In [ ]:
from agent_framework import ChatMessage, TextContent, DataContent, Role

# Load image from local file
with open("./images/joker_interrogation_scene.png", "rb") as image_file:
    image_bytes = image_file.read()

# Create a message with text and image data
message = ChatMessage(
    role = Role.USER,
    contents = [
        TextContent(text = "Tell me about the incident in this image."),
        DataContent(
            data = image_bytes,
            media_type = "image/png"
        )
    ]
)

# Creating an async function for the entire image analysis run
async def analyze_image():
    response = await agent.run(message)
    print(f"Agent: {response.text}")

# invoke the async function
await analyze_image()